Okay this if for noting : 
    our project goal is to scrap product label, price, descriptin and evaluation

My first goal is to specify how to search for the accual product because as you may know if your searching for a specific product searching by its name may give you random result like if you tap in the search par PC you may get in the result a PC support for example.
lucky for us their is an option in AMAZON which filters the results results by catalog
So all i had to do is get into the page directly using your typical browser then copy the link which contains in the header all the informations needed about your product after that you can start your coding my fghiend.

This is the link that we'll be working our starting point https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&ds=v1%3A1UlRBmA3DTPGv8k%2FdC8o92iLhOYwmXL1F8TBpveBrQs&qid=1666031082&rnid=17923671011&ref=sr_nr_n_3

first thing import necessary libraries that we'll be working with

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

Then we load the driver

In [2]:
# The option variable is to store more options before calling the driver
option = webdriver.ChromeOptions()
#option.add_argument('headless')

#call the driver 
driver = webdriver.Chrome('/home/meedbek/DATA/UBUNTU/chromedriver',options=option)

/tmp/ipykernel_59243/2080977985.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/meedbek/DATA/UBUNTU/chromedriver',options=option)


Open the link

In [3]:
#now we get into the link 

link = "https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&ds=v1%3A1UlRBmA3DTPGv8k%2FdC8o92iLhOYwmXL1F8TBpveBrQs&qid=1666031082&rnid=17923671011&ref=sr_nr_n_3"

driver.get(link)
#we ll loading directly to the page were the graphics cards are listed

Tthis is an instance of the WebDriverWait object, this object will allow to wait for pages to load or certain html tags to appear before continuing the work.

In [4]:
wait = WebDriverWait(driver,60) #the 60 refers to the timeout variable before stoping cause we can't wait for pages to load for infinite time
wait5 = WebDriverWait(driver, 5)

This is so straightforward after loading the page we need to search for all the links  then iterate into each one of them click to link get your data get back and your closely done

In [5]:
#this will wait for the presence of elements we re locating using XPATH
xpath_links = "//a[contains(@class, 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')]"
graphics = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath_links)))

In [6]:
len(graphics)

63

we have noticed that the  number of graphic card returned by the graphics variable does not match the number of the graphic card in the website page so in the script below we'll be checking if any of the links returned are duplicated or their are links that we cannot see on the page

In [7]:
dictionary = {}
for graphic in graphics:
    key = graphic.get_attribute('href') 
    if key in dictionary.keys():
        dictionary[key] = dictionary[key] + 1
    else:
        dictionary[key] = 1
        
print(dictionary.values())
print(len(dictionary.values()))

dict_values([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])
33


as we predicted we find most of the links found dublicted but the number distinct link does indeed match the number found on the page so we ll be iterating through this distinct list or dictionary

on thing we can do is go into each of the links, get informations needed about the product get back again, then when all the products were seen on one page we can go to the next one.
But this not optimal we can get all the links at one from all the pages and when we're done then we go through each link one by one

In [8]:
#checking how many of the buttons does have this class
nextButton = "//a[contains(@class, 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator')]"
len(driver.find_elements(By.XPATH,nextButton))

1

In [9]:
dictionary = {}

In [10]:
#now we save all links
done = False
while(not done):
    wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath_links)))
    time.sleep(1)
    graphics = driver.find_elements(By.XPATH,xpath_links)
    for graphic in graphics:
        key = graphic.get_attribute('href') 
        if key in dictionary.keys():
            dictionary[key] = dictionary[key] + 1
        else:
            dictionary[key] = 1
    try :
        nextp = wait5.until(EC.presence_of_element_located((By.XPATH, nextButton)))
    except :
        nextp = None
    if nextp != None :
        nextp.click()
    else :
        done = True 

In [11]:
#check how many link found
len(dictionary)

1455

we got the links now we save them into a json file

but before doing so we're gonna reset all the values of the dict to 0 this will be helpfull afterwards

In [14]:
links = dictionary

In [15]:
for link in list(links.keys()):
    links[link] = 0

In [16]:
import json

In [17]:
with open("links.json", "w") as fp:
    json.dump(dictionary,fp)

This is a new section after getting all links its time get into each of the links and get all the inforamtions

**This is the final code**

Read the json file we saved earlier

In [18]:
with open('links.json') as json_file:
   links = json.load(json_file)

If the scraping stopped for some reason you can load newlinks.json instead this link so that you don't from start all over again

In [19]:
# with open('newlinks.json') as json_file:
#     links = json.load(json_file)

To understand more links.json is a list of keys values, the keys represent each link and the value represent if it was visited or not so each time the scrapping stops for a reason or another you can continue from where started

In [20]:
#this function return the xpath of any html tag given, the className is a list of tags in form of a String
def xpath(className, context=False):
    if context==True:
        return ".//*[contains(@class,'"+className+"')]"
    return "//*[contains(@class,'"+className+"')]"

In [21]:
#this allows to retreive the rating from the text their is an example below
def retreiveRating(text):    
    if text == None:
        return None
    digits = [ i for i in text if i.isdigit()]
    if len(digits) == 2:
        return digits[0]+'.'+digits[1]
    return digits[0]

In [22]:
retreiveRating("the rating is 4-5")

'4.5'

We define the classes of each element

In [23]:
labelClass = "a-size-large product-title-word-break"
priceClass = "a-price-whole"
price2 = "a-price-fraction"

descriptionID = "productDescription" #value is in p/span
ratingClass = "a-size-medium a-color-base" 

buttonReviewsClass = "a-link-emphasis a-text-bold"

reviewClass = "a-section celwidget"
reviewTitleClass = "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"

numberOfStarsClass = "a-icon-alt"

reviewInforamtionClass = "a-size-base a-color-secondary review-date"

reviewContentClass = "a-size-base review-text review-text-content" #value span

we just do some necessary initiations like the variable that will the data and the driver etc

In [24]:
data = {}
data['data'] = []
listOfGraphics = data['data']

keys = list(links.keys())

option = webdriver.ChromeOptions()
#option.add_argument('headless')
driver = webdriver.Chrome('/home/meedbek/DATA/UBUNTU/chromedriver', options=option)
wait = WebDriverWait(driver,20) 
wait5 = WebDriverWait(driver, 0.5)

/tmp/ipykernel_59243/59067237.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/meedbek/DATA/UBUNTU/chromedriver', options=option)


Start Scrapping

In [ ]:
counter = 0
for i in range(len(keys)) :
    print("Counter :", counter)
    if(links[keys[i]] == 1):
        counter += 1
        continue 
    if(counter%100 == 0):    
        counter == 0
        with open("newlinks.json", "w") as fp:
            json.dump(links,fp)
        with open ("data.json", "w") as fp:
            json.dump(data,fp)
    
    driver.get(keys[i])
    label = wait.until(EC.presence_of_element_located((By.XPATH, xpath(labelClass)))).text
    try :
        price = wait5.until(EC.presence_of_element_located((By.XPATH, xpath(priceClass)))).text
    except : 
        price = None
    try :
        cents = wait5.until(EC.presence_of_element_located((By.XPATH, xpath(price2)))).text
    except : 
        cents = None
    try : 
        descr = wait5.until(EC.presence_of_element_located((By.ID, descriptionID)))
        description = descr.find_element_by_xpath("p/span").text
    except :
        description = None
    try :
        rating = wait5.until(EC.presence_of_element_located((By.XPATH, xpath(ratingClass)))).text.split(' ')[0]
    except : 
        rating = None
    
    try :
        prix = price+'.'+cents
        if('..' in prix):
            prix = '.'.join(prix.split('..'))
    except :
        prix = None 
        
    listOfGraphics.append({'label' : label, 'price' : prix,'description' : description, 'rating' : rating})
    try :
        wait5.until(EC.presence_of_element_located((By.XPATH, xpath(buttonReviewsClass)))).click()
        reviews = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath(reviewClass))))
        listOfGraphics[-1]['Reviews'] = []
        RE = listOfGraphics[-1]['Reviews']
        for j in range(len(reviews)) :
            waitR = WebDriverWait(reviews[j],0.5) 
            try :
                reviewTitle = waitR.until(EC.presence_of_element_located((By.XPATH, xpath(reviewTitleClass,context=True)))).text 
            except :
                reviewTitle = None
            try : 
                reviewRating = waitR.until(EC.presence_of_element_located((By.TAG_NAME, 'i'))).get_attribute('class') 
            except : 
                reviewRating = None
            try : 
                reviewInforamtion = waitR.until(EC.presence_of_element_located((By.XPATH, xpath(reviewInforamtionClass,context=True)))).text 
            except : 
                reviewInforamtion = None 
            try : 
                reviewContent = waitR.until(EC.presence_of_element_located((By.XPATH, xpath(reviewContentClass,context=True)))).find_element_by_tag_name('span').text  
            except :
                reviewContent = None
            RE.append({'title' : reviewTitle, 'rating' : retreiveRating(reviewRating), 'location-date': reviewInforamtion, 'text' : reviewContent})
    except : 
        continue
    finally :
        counter += 1
        links[keys[i]] = 1 
with open("newlinks.json", "w") as fp:
            json.dump(links,fp)
with open ("dataEnd.json", "w") as fp:
    json.dump(data,fp)

Counter : 0


/media/meedbek/DATA/UBUNTU/scrap_env/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/media/meedbek/DATA/UBUNTU/scrap_env/lib/python3.10/site-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


Counter : 1
Counter : 2
Counter : 3
Counter : 4
Counter : 5
Counter : 6
Counter : 7
Counter : 8
Counter : 9
Counter : 10
Counter : 11
Counter : 12
Counter : 13
Counter : 14
Counter : 15


In [ ]:
len(data['data'])

In [ ]:
len(links)
rest = 0
for value in links.values():
    if value == 1:
        rest += 1
rest

Now afer we had all this files we need to merge all of them

In [ ]:
files = []
files.append('data823')
files.append('data125')
files.append('data5')
files.append('dataEnd')

In [ ]:
final = []

In [ ]:
for file in files:
    with open(file+'.json') as json_file:
        final = final + json.load(json_file)['data']

In [ ]:
len(final)

In [ ]:
dicti = {'data' : final}
with open ("completeData.json", "w") as fp:
    json.dump(dicti,fp)